# AI Tools

## Project - Airlines AI Assistant

We will make an AI Customer Support assistant for an Airline with pre-fixed prices

In [1]:
# Importing the Libraries

import requests
import json
import time
import re
import ollama
from typing import List
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup
import gradio as gr
import uuid

In [2]:
# Initializing System Messages and ollama details

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

# system_message += """
# TOOLS INSTRUCTIONS:
# - Only call a tool if the user explicitly asks about flight booking, ticket prices, or related information.
# - If the user is greeting, chatting, or asking a general question, DO NOT call a tool. Just respond normally.
# """

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME="qwen3"

In [3]:
# Chat function for gradio

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "temperature": 0.8,
        "stream": True  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=True)
    result=""
    try:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    delta = data.get("message", {}).get("content", "")
                    if delta:
                        # Optional: remove unwanted <think> tags or others
                        clean_delta = re.sub(r"<think>.*?</think>", "", delta, flags=re.DOTALL)
                        result += clean_delta
                        yield result
                except json.JSONDecodeError:
                    continue
    finally:
        # Ensure generator exits cleanly
        yield result

In [4]:
# Gradio UI

gr.ChatInterface(fn=chat, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### Tools
Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

In [5]:
# Let's start by making a useful function

ticket_prices = {"chennai": "₹ 399", "mumbai": "₹ 899", "bangalore": "₹ 400", "hyderabad": "₹ 499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    return ticket_prices.get(destination_city.lower(), "Unknown City")

In [6]:
get_ticket_price("Chennai")

Tool get_ticket_price called for Chennai


'₹ 399'

In [7]:
get_ticket_price("London")

Tool get_ticket_price called for London


'Unknown City'

In [8]:
# There's a particular dictionary structure that's required to describe our function:

# Tool Name
price_function = {
        "name": "get_ticket_price",  # Name of the function
        # This is important since its passed to LLM so that when it should call the tool we have to mention it clearly, its like system prompt for tools. Be clear as possible 
        "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
        "parameters": {
            # Here we have to give the details of the parameters, here we have one parameter so mentioning that here
            "type": "object",
            "properties":{
                "destination_city":{
                    # Mentioning the data type and where and how this input is used
                    "type": "string",
                    "description": "The city that the customer wants to travel to",
                },
            },
            # we have to mention what are the must required parameters of the function(tool)
            "required":["destination_city"],
            # It restricts the input object to allow only the explicitly defined properties — no extras.
            # like only "destination_city": "Berlin" this is allowed they can't send extra prameters like "class": "economy"
            "additionalProperties": False,
        }
    }

In [9]:
# And this is included in a list of tools:

tools = [
    {
        "type": "function",  # Since we are calling the function we have to mention it
        "function": price_function,
    }
]

In [10]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message["tool_calls"][0] # Since we are using only one tool we are getting the first one
    arguments = tool_call["function"]["arguments"]
    if isinstance(arguments, str):
        arguments = json.loads(arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)

    tool_call_id = tool_call.get("id", str(uuid.uuid4()))
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city, "price": price}),
        "tool_call_id": tool_call_id
    }
    return response, city

In [11]:
# For tools we are going to use ollama package instead of APIs(Since APIs don't support tools)
# This tools support will be provides by only specific models so we switch to Qwen from Mistral

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    result = ""
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=messages,
            tools=tools,
            stream=False,
            options={
                "temperature": 0.8
            }
        )
        print(response, "\n")

        msg = response.get("message", {})
        if msg.get("tool_calls"):
            print(msg.get("tool_calls"))
            tool_response, city = handle_tool_call(msg)
            messages.append(msg)
            messages.append(tool_response)
            response = ollama.chat(
                model=MODEL_NAME,
                messages=messages,
                stream=False,
                options={
                    "temperature": 0.8
                }
            )

            msg = response.get("message", {})
            
        if msg.get("content"):
            return {"role": "assistant", "content": msg["content"]}
        
        return {"role": "assistant", "content": "I'm not sure how to respond."}
                    
    except Exception as e:
        print("Exception: ", repr(e))

In [12]:
# Chat Interface

gr.ChatInterface(fn=chat, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


### Multi Tool calling

Even though the tool calling is working fine, we can call only one tools at a time since we are fetching the first tool
```
tool_call = message["tool_calls"][0] # Since we are using only one tool we are getting the first one
```
from the ```handle_tool_call``` function and the LLM can call only one Tool at a time because of 
```
if msg.get("tool_calls"):
```
from ```chat``` function

Now we are going to replace it with ```for and while``` loops for multiple tool calling option 

In [13]:
# We are handling all tool calls in a loop so its all handled

def multi_handle_tool_calls(message):
    responses = [] # using list to append all tools responses
    cities = [] # using list to append all cities fetched from tool calls
    for tool_call in message["tool_calls"]:
        arguments = tool_call["function"]["arguments"]
        if isinstance(arguments, str):
            arguments = json.loads(arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
    
        tool_call_id = tool_call.get("id", str(uuid.uuid4()))
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city, "price": price}),
            "tool_call_id": tool_call_id
        }
        responses.append(response)
        cities.append(city)
    return responses, cities

In [14]:
# This function will make multiple calls

def multi_chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    result = ""
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=messages,
            tools=tools,
            stream=False,
            options={
                "temperature": 0.8
            }
        )
        print(response, "\n")

        msg = response.get("message", {})
        while msg.get("tool_calls"): # this will allow the LLM to run all tool calls it makes
            print(msg.get("tool_calls"))
            tool_response, city = multi_handle_tool_calls(msg)
            messages.append(msg)
            messages.extend(tool_response) # we are recieving a list of response so extend it instead of appending it
            response = ollama.chat(
                model=MODEL_NAME,
                messages=messages,
                stream=False,
                options={
                    "temperature": 0.8
                }
            )

            msg = response.get("message", {})
            
        if msg.get("content"):
            return {"role": "assistant", "content": msg["content"]}
        
        return {"role": "assistant", "content": "I'm not sure how to respond."}
                    
    except Exception as e:
        print("Exception: ", repr(e))

In [15]:
# Chat Interface

gr.ChatInterface(fn=multi_chat, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


### Now we will connect the DB to fetch the values for the Tools to run

In [46]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

DB_HOST="localhost"
DB_PORT="3306"
DB_NAME="personal_DB"
DB_USER="Alex"
DB_PASSWORD="Alex@$14798|<</>>"

connection_string = (
    f"mysql+pymysql://{DB_USER}:{quote_plus(DB_PASSWORD)}@"
    f"{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

engine = create_engine(connection_string, echo=True)

In [47]:
# with engine.connect() as conn:
#     result = conn.execute(text("select * from personal_DB.prices;"))

In [50]:
def get_ticket_price_db(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with engine.connect() as conn:
        result = conn.execute(text(f'SELECT price FROM personal_DB.prices WHERE city = "{city.lower()}"'))
        result = result.fetchall()
        conn.close()
        return f"Ticket price to {city} is ₹ {result[0][0]}" if result else "No price data available for this city"

In [51]:
get_ticket_price_db("Chennai")

DATABASE TOOL CALLED: Getting price for Chennai
2025-11-30 19:11:54,539 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-30 19:11:54,539 INFO sqlalchemy.engine.Engine SELECT price FROM personal_DB.prices WHERE city = "chennai"
2025-11-30 19:11:54,540 INFO sqlalchemy.engine.Engine [cached since 20.46s ago] {}
2025-11-30 19:11:54,541 INFO sqlalchemy.engine.Engine ROLLBACK


'Ticket price to Chennai is ₹ 399.0'

In [56]:
def multi_handle_tool_calls_db(message):
    responses = [] # using list to append all tools responses
    cities = [] # using list to append all cities fetched from tool calls
    for tool_call in message["tool_calls"]:
        if tool_call["function"]["name"] == "get_ticket_price":
            arguments = tool_call["function"]["arguments"]
            if isinstance(arguments, str):
                arguments = json.loads(arguments)
            city = arguments.get('destination_city')
            price = get_ticket_price_db(city)
        
            tool_call_id = tool_call.get("id", str(uuid.uuid4()))
            response = {
                "role": "tool",
                "content": json.dumps({"destination_city": city, "price": price}),
                "tool_call_id": tool_call_id
            }
            responses.append(response)
            cities.append(city)
    return responses, cities

In [57]:
def multi_chat_db(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    result = ""
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=messages,
            tools=tools,
            stream=False,
            options={
                "temperature": 0.8
            }
        )
        print(response, "\n")

        msg = response.get("message", {})
        while msg.get("tool_calls"): # this will allow the LLM to run all tool calls it makes
            print(msg.get("tool_calls"))
            tool_response, city = multi_handle_tool_calls_db(msg)
            messages.append(msg)
            messages.extend(tool_response) # we are recieving a list of response so extend it instead of appending it
            response = ollama.chat(
                model=MODEL_NAME,
                messages=messages,
                stream=False,
                options={
                    "temperature": 0.8
                }
            )

            msg = response.get("message", {})
            
        if msg.get("content"):
            return {"role": "assistant", "content": msg["content"]}
        
        return {"role": "assistant", "content": "I'm not sure how to respond."}
                    
    except Exception as e:
        print("Exception: ", repr(e))

In [58]:
# Chat Interface

gr.ChatInterface(fn=multi_chat_db, chatbot=gr.Chatbot(type="messages")).launch()

/home/alexender/Desktop/Projects/My_projects/envs/Torch_env/lib/python3.12/site-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


model='qwen3' created_at='2025-11-30T14:18:21.481230367Z' done=True done_reason='stop' total_duration=24245003308 load_duration=6305198128 prompt_eval_count=226 prompt_eval_duration=3092597310 eval_count=70 eval_duration=14660346451 message=Message(role='assistant', content='Hello! How can I assist you today?', thinking='Okay, the user said "hello". I need to respond politely and briefly. Since there\'s no specific query here, I should just greet them back and offer assistance. No need to call any functions because they didn\'t ask for anything yet. Keep it friendly and open-ended.\n', images=None, tool_calls=None) 

model='qwen3' created_at='2025-11-30T14:19:57.568050076Z' done=True done_reason='stop' total_duration=31791957382 load_duration=246691616 prompt_eval_count=253 prompt_eval_duration=894665865 eval_count=139 eval_duration=30538808831 message=Message(role='assistant', content='', thinking="Okay, the user is asking for ticket prices for Mumbai and Chennai. Let me check the too